In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("C:\\Users\\krishnendu\\Desktop\\sample_project\\100 days of ML\\EDA\\train.csv")

In [3]:
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import ColumnTransformer

In [4]:
import numpy as np

In [5]:
df.dropna(inplace=True)

In [6]:
df.shape

(183, 12)

In [7]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [8]:
df = df[['Age','Fare','Survived']]

In [9]:
df

,Age,Fare,Survived
1,38.0,71.2833,1
3,35.0,53.1000,1
6,54.0,51.8625,0
10,4.0,16.7000,1
11,58.0,26.5500,1
...,...,...,...
871,47.0,52.5542,1
872,33.0,5.0000,0
879,56.0,83.1583,1
887,19.0,30.0000,1


In [15]:
x = df.iloc[:,:2]
y = df.iloc[:,-1]

In [16]:
x

,Age,Fare
1,38.0,71.2833
3,35.0,53.1000
6,54.0,51.8625
10,4.0,16.7000
11,58.0,26.5500
...,...,...
871,47.0,52.5542
872,33.0,5.0000
879,56.0,83.1583
887,19.0,30.0000


In [17]:
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [18]:
clf = DecisionTreeClassifier()

In [19]:
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

In [20]:
accuracy_score(y_test,y_pred)

0.6486486486486487

In [46]:
np.mean(cross_val_score(DecisionTreeClassifier(),x,y,cv=10,scoring='accuracy'))


0.6619883040935673

In [47]:
kbin_age = KBinsDiscretizer(n_bins = 15,encode='ordinal',strategy='quantile')
kbin_fare = KBinsDiscretizer(n_bins= 15, encode='ordinal', strategy = 'quantile')

In [48]:
trf = ColumnTransformer([
    ('first',kbin_age,[0]),
    ('second',kbin_fare,[1])
])

In [49]:
X_train_trf = trf.fit_transform(X_train)
X_test_trf = trf.fit_transform(X_test)

In [50]:
trf.named_transformers_['first'].bin_edges_

array([array([ 0.92,  8.8 , 18.6 , 23.2 , 24.  , 27.  , 31.4 , 37.6 , 39.2 ,
              40.6 , 43.  , 47.4 , 49.6 , 51.2 , 56.8 , 71.  ])             ],
      dtype=object)

In [51]:
output = pd.DataFrame({
    'age':X_train['Age'],
    'age_trf':X_train_trf[:,0],
    'fare':X_train['Fare'],
    'fare_trf':X_train_trf[:,1]
})

In [52]:
output['age_labels'] = pd.cut(x=X_train['Age'],
                                    bins=trf.named_transformers_['first'].bin_edges_[0].tolist())
output['fare_labels'] = pd.cut(x=X_train['Fare'],
                                    bins=trf.named_transformers_['second'].bin_edges_[0].tolist())



In [53]:
output

,age,age_trf,fare,fare_trf,age_labels,fare_labels
331,45.5,10.0,28.5000,4.0,"(43.0, 47.4]","(16.815, 29.502]"
336,29.0,5.0,66.6000,8.0,"(27.0, 31.4]","(62.255, 74.495]"
193,3.0,0.0,26.0000,2.0,"(0.92, 8.8]","(16.815, 29.502]"
75,25.0,4.0,7.6500,0.0,"(24.0, 27.0]",NaN
248,37.0,9.0,52.5542,6.0,"(31.4, 37.6]","(52.0, 62.255]"
...,...,...,...,...,...,...
520,30.0,5.0,93.5000,11.0,"(27.0, 31.4]","(79.65, 97.218]"
92,46.0,10.0,61.1750,7.0,"(43.0, 47.4]","(52.0, 62.255]"
460,48.0,11.0,26.5500,3.0,"(47.4, 49.6]","(16.815, 29.502]"
872,33.0,6.0,5.0000,0.0,"(31.4, 37.6]",NaN


In [54]:
clf = DecisionTreeClassifier()
clf.fit(X_train_trf,y_train)
y_pred2 = clf.predict(X_test_trf)

In [55]:
accuracy_score(y_test,y_pred2)

0.6486486486486487

In [56]:
X_trf = trf.fit_transform(x)
np.mean(cross_val_score(DecisionTreeClassifier(),X_trf,y,cv=10,scoring='accuracy'))

0.5517543859649123

In [61]:
def discretize(bins,strategy):
    kbin_age = KBinsDiscretizer(n_bins=15, encode='ordinal',strategy= strategy)
    kbin_fare = KBinsDiscretizer(n_bins=15, encode='ordinal',strategy= strategy)
    
    # Implemeny ColumnTransformer
    trf = ColumnTransformer([
        ('first',kbin_age,[0]),
        ('second',kbin_fare,[1])
    ])
    
    X_trf = trf.fit_transform(x)
    print(np.mean(cross_val_score(DecisionTreeClassifier(),X_trf,y,cv=10,scoring='accuracy')))
    
    plt.figure(figsize=(14,5))
    plt.subplot(121)
    plt.hist(x['Age'])
    plt.title('Before')
    
    plt.subplot(122)
    plt.hist(X_trf[:,0],color='red')
    plt.title("After")
    
    plt.show()
    
    plt.figure(figsize=(14,4))
    plt.subplot(121)
    plt.hist(x['Fare'])
    plt.title("Before")

    plt.subplot(122)
    plt.hist(X_trf[:,1],color='red')
    plt.title("Fare")

    plt.show()
    
    

In [77]:
discretize(5,'kmeans')

AttributeError: 'NoneType' object has no attribute 'split'